### AutoLamella Model


In [ ]:
# openfibsem-baseline-34.pt
# autolamella-02-34.pt


%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import matplotlib.pyplot as plt

TEST_DATA_PATH = "/home/patrick/github/data/autolamella/test"


filenames = glob.glob(os.path.join(TEST_DATA_PATH, "*.tif"))


print(filenames)

In [ ]:
from fibsem.segmentation.model import load_model
from fibsem.detection import detection
from fibsem.structures import FibsemImage

CHECKPOINT_BASELINE = "openfibsem-baseline-34.pt" # autoliftout-base
CHECKPOINT_ONE = "autolamella-02-34.pt"
CHECKPOINT_v3 = "autolamella-03-34.pt"

CHECKPOINTS = [CHECKPOINT_BASELINE, CHECKPOINT_ONE, CHECKPOINT_v3]

FEATURES = [detection.LamellaCentre()]

DETECTIONS = []


for fname in filenames:
    print(fname)
    img = FibsemImage.load(fname)

    dets: list[detection.DetectedFeatures] = []

    for CHECKPOINT in CHECKPOINTS:
        print(CHECKPOINT)
        model = load_model(CHECKPOINT, encoder="resnet34")
        
        det = detection.detect_features(img, model, 
                                            features=[detection.LamellaCentre()], 
                                            pixelsize=img.metadata.pixel_size.x, filter=False)
        dets.append(det)
    
    DETECTIONS.append(dets)


In [ ]:


for fname, dets in zip(filenames, DETECTIONS):

    titles = [f"{checkpoint}" for checkpoint in CHECKPOINTS]

    fig = detection.plot_detections(dets, titles=titles)

    # save figure
    # plot_fname = os.path.basename(fname).replace(".tif", ".png")
    
    # fig.savefig(plot_fname, dpi=300, bbox_inches="tight")

In [ ]:
# dataset sizes

DATA_PATH_BASE  = "/home/patrick/github/data/autolamella"
BASELINE_TRAIN_PATH = "/home/patrick/github/data/liftout/training/train/images"

PATH_TRAIN_2 = os.path.join(DATA_PATH_BASE, "train2")
PATH_TRAIN_3 = os.path.join(DATA_PATH_BASE, "train3")

files_train_2 = glob.glob(os.path.join(PATH_TRAIN_2, "*.tif"))
files_train_3 = glob.glob(os.path.join(PATH_TRAIN_3, "*.tif"))
baseline_train = glob.glob(os.path.join(BASELINE_TRAIN_PATH, "*.tif"))

print("autolamella dataset sizes")
print("baseline: ", len(baseline_train))
print("train2: ", len(files_train_2))
print("train3: ", len(files_train_3))


import plotly.express as px

# plot count on top of bars
fig = px.bar(x=["baseline (common)", "train2", "train3"], 
    y=[len(baseline_train), len(files_train_2), len(files_train_3)], 
    text=[len(baseline_train), len(files_train_2), len(files_train_3)],
    color=["baseline (common)", "train2", "train3"],
    title="AutoLamella Training Dataset Sizes")

fig.show()
